# Pengerjaan Tugas Deep Learning menggunakan PyTorch and TensorFlow (Week 7, WMT Dataset Menggunakan Encoder-to-Decoder LSTM)

## Persiapan: Instalasi library

### 1. Memastikan Instalasi library

In [2]:
# !pip install numpy matplotlib scikit-learn torch tensorflow
# %pip install numpy matplotlib scikit-learn torch tensorflow[and-cuda] keras-tuner nltk imbalanced-learn

%pip install keras-tuner imbalanced-learn datasets optuna transformers

Note: you may need to restart the kernel to use updated packages.


### 2. Mengimpor Library yang Dibutuhkan

In [3]:
# Import library yang diperlukan
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torch.nn.utils.rnn import pad_sequence
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.layers import LSTM, Bidirectional, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset
import keras_tuner as kt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import optuna
from transformers import AutoTokenizer
import time

# Memeriksa apakah GPU tersedia dan dapat digunakan oleh PyTorch
gpu_available = torch.cuda.is_available()
print(f"GPU available: {gpu_available}")

# Memeriksa apakah GPU tersedia dan dapat digunakan oleh TensorFlow
gpu_available = tf.test.is_gpu_available()
print(f"GPU available: {gpu_available}")


/home/husain/dlvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU available: True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: True


I0000 00:00:1745685251.747179  505995 gpu_device.cc:2019] Created device /device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


### 3. Mendefinisikan Parameter dan Pre-processing Dataset

In [4]:
# Konstanta dan parameter
NUM_WORDS = 30000
MAXLEN = 1000
BATCH_SIZE = 16
EPOCHS = 25

# Set seed untuk reproduktivitas
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
tf.random.set_seed(42)

# Memuat dataset
print("Loading dataset...")
dataset = load_dataset("wmt14", "de-en")
print(f"Dataset loaded with {len(dataset['train'])} training examples.")

# Take a subset for faster training
data = dataset['train'].shuffle(seed=42).select(range(NUM_WORDS))  # Use a smaller subset for faster training

german_texts = [item['translation']['de'] for item in data]
english_texts = [item['translation']['en'] for item in data]

# Tokenisasi teks
german_tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<unk>")
german_tokenizer.fit_on_texts(german_texts)
german_sequences = german_tokenizer.texts_to_sequences(german_texts)
english_tokenizer = Tokenizer(num_words=NUM_WORDS, oov_token="<unk>")
english_tokenizer.fit_on_texts(english_texts)
english_sequences = english_tokenizer.texts_to_sequences(english_texts)

# Build vocabularies
german_vocab = set(german_tokenizer.word_index.keys())
english_vocab = set(english_tokenizer.word_index.keys())
german_vocab_size = len(german_vocab)
english_vocab_size = len(english_vocab)

# Create word-to-index mappings
german_word2idx = {word: idx for idx, word in enumerate(sorted(german_vocab))}
english_word2idx = {word: idx for idx, word in enumerate(sorted(english_vocab))}

# Create index-to-word mappings
german_idx2word = {idx: word for word, idx in german_word2idx.items()}
english_idx2word = {idx: word for word, idx in english_word2idx.items()}

# Convert tokens to indices
german_indices = [[german_word2idx[word] for word in seq] for seq in german_sequences]
english_indices = [[english_word2idx[word] for word in seq] for seq in english_sequences]

# Pad sequences to the same length
german_padded = [seq + [german_word2idx['<end>']] * (MAXLEN - len(seq)) for seq in german_indices]
english_padded = [seq + [english_word2idx['<end>']] * (MAXLEN - len(seq)) for seq in english_indices]
# Convert to numpy arrays
german_data = np.array(german_padded)
english_data = np.array(english_padded)


Loading dataset...


KeyboardInterrupt: 

### 4. Menyiapkan Fungsi Pelatihan

In [ ]:
def train_model_pytorch(dataset, model, german_data, english_data, german_vocab_size, english_vocab_size, params):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        german_data, english_data, test_size=0.2, random_state=42
    )
    
    # Create datasets and dataloaders
    train_dataset = dataset(X_train, y_train)
    test_dataset = dataset(X_test, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'])
    
    # Initialize model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    
    # Training loop
    best_accuracy = 0
    history = {'accuracy': [], 'loss': [], 'val_accuracy': [], 'val_loss': []}
    for epoch in range(params['epochs']):
        model.train()
        train_loss = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            # Clear gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs, targets, teacher_forcing_ratio=params['teacher_forcing_ratio'])
            
            # Reshape outputs and targets for loss calculation
            outputs = outputs[:, 1:].reshape(-1, english_vocab_size)
            targets = targets[:, 1:].reshape(-1)
            
            # Calculate loss
            loss = criterion(outputs, targets)
            
            # Backward pass and optimize
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), params['clip'])
            optimizer.step()
            
            train_loss += loss.item()
            
            if batch_idx % 100 == 0:
                print(f'Epoch: {epoch+1}/{params["epochs"]}, Batch: {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
        
        # Evaluation and validation
        model.eval()
        all_predictions = []
        all_targets = []
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                
                outputs = model(inputs, targets, teacher_forcing_ratio=0.0)
                
                # Get predictions (exclude first token which is <start>)
                predictions = outputs[:, 1:].argmax(dim=2)
                loss = criterion(outputs, targets)

                # Reshape for evaluation
                all_predictions.extend(predictions.cpu().numpy().reshape(-1))
                all_targets.extend(targets[:, 1:].cpu().numpy().reshape(-1))

                val_loss += loss.item()
                predicted = (outputs.data > 0.5).float()
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()
        
        # Calculate metrics
        # Filter out padding tokens
        valid_indices = [i for i, target in enumerate(all_targets) if target != 0]
        valid_preds = [all_predictions[i] for i in valid_indices]
        valid_targets = [all_targets[i] for i in valid_indices]
        
        accuracy = accuracy_score(valid_preds, valid_targets)
        
        print(f'Epoch {epoch+1}/{params["epochs"]} - Validation Accuracy:')
        print(f'Accuracy: {accuracy:.4f}')

        history['accuracy'].append(accuracy)
        history['loss'].append(train_loss / len(train_loader))

        val_accuracy = val_correct / val_total
        history['val_accuracy'].append(val_accuracy)
        history['val_loss'].append(val_loss / len(test_loader))
        
        # Save the best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'best_pytorch_model.pt')
            
            if best_accuracy >= 0.9:
                print(f'Target accuracy of 90% achieved: {best_accuracy:.4f}')
    
    return model, best_accuracy, valid_preds, valid_targets

In [ ]:
# Fungsi untuk melatih model TensorFlow
def train_model_tensorflow(model, german_data, english_data, german_vocab_size, english_vocab_size, params):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        german_data, english_data, test_size=0.2, random_state=42
    )
    
    # Define callbacks
    checkpoint = ModelCheckpoint(
        'best_tf_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True,
        verbose=1
    )
    
    # Train the model
    history = model.fit(
        [X_train, y_train[:, :-1]],  # Input: encoder_input, decoder_input
        y_train[:, 1:],              # Target: decoder_target (shifted by 1)
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        validation_split=0.1,
        callbacks=[checkpoint, early_stopping]
    )
    
    # Evaluate the model
    # Get predictions on test set
    y_pred_prob = model.predict([X_test, y_test[:, :-1]])
    y_pred = np.argmax(y_pred_prob, axis=2)
    
    # Reshape for evaluation
    y_pred_flat = y_pred.reshape(-1)
    y_test_flat = y_test[:, 1:].reshape(-1)
    
    # Filter out padding tokens
    valid_indices = [i for i, target in enumerate(y_test_flat) if target != 0]
    valid_preds = [y_pred_flat[i] for i in valid_indices]
    valid_targets = [y_test_flat[i] for i in valid_indices]
    
    return model, history, valid_preds, valid_targets

### 4. Menyiapkan Fungsi Evaluasi

In [ ]:
# Fungsi untuk evaluasi model
def calculate_metrics(y_true, y_pred, y_pred_prob=None):
    # Convert one-hot encoded predictions to class labels
    if len(y_pred.shape) > 1 and y_pred.shape[1] > 1:
        y_pred_labels = np.argmax(y_pred, axis=1)
    else:
        y_pred_labels = y_pred
    
    # For multi-class metrics, we'll need to binarize
    accuracy = accuracy_score(y_true, y_pred_labels)
    
    # For binary case or with micro averaging
    try:
        precision = precision_score(y_true, y_pred_labels, average='micro')
        recall = recall_score(y_true, y_pred_labels, average='micro')
        f1 = f1_score(y_true, y_pred_labels, average='micro')
    except:
        precision = 0
        recall = 0
        f1 = 0
    
    # ROC AUC only for binary or with OvR approach
    auc_score = 0
    if y_pred_prob is not None:
        try:
            # For multiclass, we can use OvR approach
            if len(np.unique(y_true)) > 2:
                tpr, fpr, _ = roc_curve(y_true, y_pred_prob[:, 1])
                auc_score = roc_auc_score(tf.keras.utils.to_categorical(y_true), y_pred_prob, multi_class='ovr')
            else:
                fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
                auc_score = roc_auc_score(y_true, y_pred_prob[:, 1] if y_pred_prob.shape[1] > 1 else y_pred_prob)
        except:
            tpr, fpr, _ = 0
            auc_score = 0
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc_score,
        'tpr': tpr,
        'fpr': fpr
    }

### 5. Menyiapkan Fungsi Visualisasi

In [ ]:
# Fungsi untuk membuat visualisasi hasil
# Fungsi untuk membuat visualisasi training history
def plot_training_history(history, model_name):
    plt.figure(figsize=(12, 5))
    
    best_epoch = history['val_loss'].index(min(history['val_loss']))
    best_val_loss = min(history['val_loss'])

    # Plot Loss, dengan tanda pada titik terbaik
    plt.subplot(1, 2, 1)
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Valid Loss')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label='Best Model')
    plt.axhline(y=best_val_loss, color='g', linestyle='--', label='Best Val Loss')
    plt.title(f'{model_name} - Loss History')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history['accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Valid Accuracy')
    plt.axvline(x=best_epoch, color='r', linestyle='--', label='Best Model')
    plt.axhline(y=history['val_accuracy'][best_epoch], color='g', linestyle='--', label='Val Accuracy at Best Model')
    plt.title(f'{model_name} - Accuracy History')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(f'training_history_tensorflow_{model_name}.png')
    plt.show()

# Fungsi untuk membuat confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'{model_name} - Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Negative', 'Positive'])
    plt.yticks(tick_marks, ['Negative', 'Positive'])
    
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_tensorflow_{model_name}.png')
    plt.show()

# Fungsi untuk membuat ROC curve
def plot_roc_curve(results_dict):
    plt.figure(figsize=(10, 8))
    
    for model_name, result in results_dict.items():
        plt.plot(result['fpr'], result['tpr'], label=f'{model_name} (AUC = {result["auc"]:.4f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend()
    plt.grid(True)
    plt.savefig('roc_curve_comparison_tensorflow.png')
    plt.show()

# Fungsi untuk membuat bar chart perbandingan metrik
def plot_metrics_comparison(results_dict):
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    models = list(results_dict.keys())
    
    values = {metric: [results_dict[model][metric] for model in models] for metric in metrics}
    
    plt.figure(figsize=(12, 8))
    bar_width = 0.15
    index = np.arange(len(models))
    
    for i, metric in enumerate(metrics):
        plt.bar(index + i * bar_width, values[metric], bar_width, 
                label=metric.capitalize())
    
    plt.xlabel('Models')
    plt.ylabel('Scores')
    plt.title('Performance Metrics Comparison')
    plt.xticks(index + bar_width * 2, models)
    plt.legend()
    plt.grid(True, axis='y')
    plt.savefig('metrics_comparison_tensorflow.png')
    plt.show()


## Model Encoder-to-Decoder LSTM Menggunakan PyTorch

### 1. Penyusunan Dataset dan Loader untuk PyTorch

In [ ]:
# Custom PyTorch dataset
class TranslationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = torch.tensor(inputs, dtype=torch.long)
        self.targets = torch.tensor(targets, dtype=torch.long)
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

### 2. Menyusun Model Encoder-Decoder LSTM


In [ ]:
# Model Encoder-Decoder PyTorch
class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_vocab_size, output_vocab_size, embed_size, hidden_size, num_layers, dropout=0.2):
        super(EncoderDecoderLSTM, self).__init__()
        
        # Encoder
        self.encoder_embedding = nn.Embedding(input_vocab_size, embed_size)
        self.encoder_lstm = nn.LSTM(embed_size, hidden_size, num_layers, 
                                   batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        # Decoder
        self.decoder_embedding = nn.Embedding(output_vocab_size, embed_size)
        self.decoder_lstm = nn.LSTM(embed_size, hidden_size, num_layers, 
                                   batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.output_layer = nn.Linear(hidden_size, output_vocab_size)
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.output_layer.out_features
        
        # Tensor to store outputs
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(source.device)
        
        # Encode the source sequence
        encoder_embedded = self.encoder_embedding(source)
        _, (hidden, cell) = self.encoder_lstm(encoder_embedded)
        
        # First input to the decoder is the <start> token
        decoder_input = target[:, 0].unsqueeze(1)
        
        for t in range(1, target_len):
            # Use previous hidden and cell states
            decoder_embedded = self.decoder_embedding(decoder_input)
            decoder_output, (hidden, cell) = self.decoder_lstm(decoder_embedded, (hidden, cell))
            prediction = self.output_layer(decoder_output)
            
            # Store prediction
            outputs[:, t, :] = prediction.squeeze(1)
            
            # Teacher forcing
            use_teacher_forcing = random.random() < teacher_forcing_ratio
            if use_teacher_forcing:
                decoder_input = target[:, t].unsqueeze(1)
            else:
                # Get the highest predicted token
                top1 = prediction.argmax(2)
                decoder_input = top1
                
        return outputs

### 3. Menyusun Fungsi Hyperparameter Tuning

In [ ]:
def objective_pytorch(trial, framework, german_data, english_data, german_vocab_size, english_vocab_size):
    # Common hyperparameters
    params = {
        'embed_size': trial.suggest_int('embed_size', 128, 512, step=64),
        'hidden_size': trial.suggest_int('hidden_size', 128, 512, step=64),
        'num_layers': trial.suggest_int('num_layers', 1, 3),
        'batch_size': trial.suggest_int('batch_size', 16, 128, step=16),
        'epochs': 10  # Limit epochs for tuning
    }
    
    # PyTorch specific hyperparameters
    params.update({
        'dropout': trial.suggest_float('dropout', 0.1, 0.5, step=0.1),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'teacher_forcing_ratio': trial.suggest_float('teacher_forcing_ratio', 0.5, 1.0, step=0.1),
        'clip': trial.suggest_float('clip', 0.1, 5.0, step=0.1)
    })
    
    _, accuracy, _ = train_model_pytorch(
        german_data, english_data, german_vocab_size, english_vocab_size, params
    )
    
    return accuracy

# Hyperparameter tuning for PyTorch model
print("\nPerforming hyperparameter tuning for PyTorch model...")
study_pytorch = optuna.create_study(direction='maximize')
study_pytorch.optimize(
    lambda trial: objective_pytorch(trial, 'pytorch', german_data, english_data, german_vocab_size, english_vocab_size),
    n_trials=10  # Adjust as needed
)

best_params_pytorch = study_pytorch.best_params
print(f"Best PyTorch parameters: {best_params_pytorch}")

### 4. Melatih Model dengan Hyperparameter Terbaik

In [ ]:
# Add necessary parameters that are not part of the tuning
best_params_pytorch.update({
    'epochs': 20  # Full training with more epochs
})

# Train the best PyTorch model
print("\nTraining the best PyTorch model...")
model_pytorch, pytorch_accuracy, predictions_pytorch, targets_pytorch = train_model_pytorch(
    german_data, english_data, german_vocab_size, english_vocab_size, best_params_pytorch
)

### 5. Mengevaluasi Model

In [ ]:
# Load model terbaik dan evaluasi
model_pytorch.load_state_dict(torch.load('final_model_pytorch.pt'))

# Evaluate the model
results_pytorch = calculate_metrics(targets_pytorch, predictions_pytorch)
# Print results
print("PyTorch Model Evaluation Results:")
for metric, value in results_pytorch.items():
    print(f"{metric.capitalize()}: {value:.4f}")

### 6. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plot training history
plot_training_history(history_pytorch, 'pytorch')

# Plot confusion matrix
plot_confusion_matrix(targets_pytorch, predictions_pytorch, 'pytorch')

### 7. Menyimpan Model

In [ ]:
# Menyimpan model
#torch.save(final_model_pytorch.state_dict(), 'final_model_pytorch.pt')
#print("Model disimpan sebagai 'final_model_pytorch.pt'.")

## Model Encoder-to-Decoder LSTM Menggunakan TensorFlow

### 1. Konversi Dataset ke format TensorFlow

In [ ]:
# Fungsi untuk mengkonversi dataset ke format TensorFlow
def prepare_tf_dataset(dataset, batch_size=32, buffer_size=10000):
    def gen():
        for i in range(len(dataset)):
            yield {
                'encoder_inputs': dataset[i]['translation']['de'],
                'decoder_inputs': dataset[i]['translation']['en']
            }
    
    def map_func(item):
        return {
            'encoder_inputs': item['encoder_inputs'],
            'decoder_inputs': item['decoder_inputs'][:-1]  # Input tanpa token akhir
        }, item['decoder_inputs'][1:]  # Target dimulai dari token kedua
    
    ds = tf.data.Dataset.from_generator(
        gen,
        output_signature={
            'encoder_inputs': tf.TensorSpec(shape=(None,), dtype=tf.int32),
            'decoder_inputs': tf.TensorSpec(shape=(None,), dtype=tf.int32)
        }
    )
    
    ds = ds.map(map_func)
    ds = ds.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return ds

### 2. Menyusun Model Encode-to-Decoder LSTM


In [ ]:
# TensorFlow Encoder-Decoder LSTM model
def create_tf_model(input_vocab_size, output_vocab_size, embedding_dim, hidden_dim, num_layers):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    
    encoder = encoder_embedding
    encoder_states = []
    
    for i in range(num_layers):
        encoder_lstm = LSTM(hidden_dim, return_sequences=(i < num_layers-1), return_state=True,
                           name=f'encoder_lstm_{i}')
        if i == 0:
            encoder_outputs, state_h, state_c = encoder_lstm(encoder)
        else:
            encoder_outputs, state_h, state_c = encoder_lstm(encoder_outputs)
        encoder_states.append([state_h, state_c])
    
    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
    
    decoder_lstms = []
    decoder_outputs = decoder_embedding
    
    for i in range(num_layers):
        decoder_lstm = LSTM(hidden_dim, return_sequences=True, return_state=True,
                           name=f'decoder_lstm_{i}')
        decoder_lstms.append(decoder_lstm)
    
    # Connect all LSTM layers
    for i in range(num_layers):
        if i == 0:
            decoder_outputs, _, _ = decoder_lstms[i](
                decoder_outputs, initial_state=encoder_states[i])
        else:
            decoder_outputs, _, _ = decoder_lstms[i](decoder_outputs)
    
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

### 2. Menyusun Fungsi Hyperparameter Tuning

In [ ]:
def objective_tensorflow(trial, framework, german_data, english_data, german_vocab_size, english_vocab_size):
    # Common hyperparameters
    params = {
        'embed_size': trial.suggest_int('embed_size', 128, 512, step=64),
        'hidden_size': trial.suggest_int('hidden_size', 128, 512, step=64),
        'num_layers': trial.suggest_int('num_layers', 1, 3),
        'batch_size': trial.suggest_int('batch_size', 16, 128, step=16),
        'epochs': 10  # Limit epochs for tuning
    }
    
    # TensorFlow specific hyperparameters
    params.update({
        'dropout': trial.suggest_float('dropout', 0.1, 0.5, step=0.1),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    })
    
    _, metrics, _ = train_model_tensorflow(
        german_data, english_data, german_vocab_size, english_vocab_size, params
    )
    accuracy = metrics['accuracy']
    
    return accuracy

# Train the best PyTorch model
print("\nTraining the best PyTorch model...")
best_pytorch_model, pytorch_accuracy, pytorch_metrics = train_model_tensorflow(
    german_data, english_data, german_vocab_size, english_vocab_size, best_params_pytorch
)

# Hyperparameter tuning for TensorFlow model
print("\nPerforming hyperparameter tuning for TensorFlow model...")
study_tf = optuna.create_study(direction='maximize')
study_tf.optimize(
    lambda trial: objective_tensorflow(trial, 'tensorflow', german_data, english_data, german_vocab_size, english_vocab_size),
    n_trials=10  # Adjust as needed
)

best_params_tf = study_tf.best_params
print(f"Best TensorFlow parameters: {best_params_tf}")

### 4. Melatih Model dengan Hyperparameter Terbaik

In [ ]:
# Add necessary parameters that are not part of the tuning
best_params_tf.update({
    'epochs': 20  # Full training with more epochs
})

# Train the best TensorFlow model
print("\nTraining the best TensorFlow model...")
best_tf_model, history_tf, predictions_tf, targets_tf = train_model_tensorflow(
    german_data, english_data, german_vocab_size, english_vocab_size, best_params_tf
)

### 3. Mengevaluasi Model dengan Menghitung Akurasi, Presisi, Recall, F1Squared, ROC, dan AUC-ROC

In [ ]:
# Load model terbaik dan evaluasi
best_tf_model.load_weights('best_tf_model.h5')

# Evaluate the model
results_tf = calculate_metrics(targets_tf, predictions_tf)
# Print results
print("TensorFlow Model Evaluation Results:")
for metric, value in results_tf.items():
    print(f"{metric.capitalize()}: {value:.4f}")

### 4. Memvisualisasikan Prediksi dari model yang telah dilatih

In [ ]:
# Plotting hasil pelatihan
plot_training_history(history_tf.history, 'tensorflow')

# Plotting confusion matrix
plot_confusion_matrix(targets_tf, predictions_tf, 'tensorflow')

### 5. Menyimpan Model

In [ ]:
# Menyimpan model
best_tf_model.save('model_tf.keras')
print("Model disimpan sebagai 'model_tf.keras'.")

## Perbandingan antara model

In [ ]:
# Menghitung perbandingan untuk semua model
results_dict = {
    'PyTorch': results_pytorch,
    'TensorFlow': results_tf
}

# Plotting ROC curve
plot_roc_curve(results_dict)
# Plotting perbandingan metrik
plot_metrics_comparison(results_dict)